In [1]:
import pandas as pd
import numpy as np
import os
import requests
import json
from urllib.parse import urlencode

In [2]:
os.getcwd()

'C:\\Users\\alaev\\report_sek'

In [3]:
path_file = f'{os.getcwd()}\\df_sek.xlsx'
path_file_mass = f'{os.getcwd()}\\mass.xlsx'

In [4]:
df = pd.read_excel(path_file, skiprows=[1])
mass = pd.read_excel(path_file_mass)

In [5]:
df.shape

(94229, 36)

In [6]:
mass.shape

(139129, 1)

In [7]:
mass.head()

,Номер заявки
0,И-22-00-599522/522/Ю8
1,И-22-00-599525/522/Ю8
2,И-22-00-599526/522/Ю8
3,И-22-00-599528/522/Ю8
4,И-22-00-599529/522/Ю8


In [8]:
df.columns

Index(['№ п/п', 'Филиал', 'РЭС', 'Дата выполнения задания',
       'Дата подтверждения задания Подрядчиком',
       'Дата подтверждения задания РМР', 'Дата отклонения задания РМР',
       'Пользователь в "Цифровой контролёр"', 'Категория потребителей',
       'Вид работ:', 'Объект работ', 'Адрес потребителя', 'Сетевой адрес',
       'ФИО', 'Номер заявки SAP', 'Дата заяки SAP',
       'Дата поступления заявки в МЛ', 'Текущий статус\n',
       'Причина отклонения', '№ действующего/ снятого ПУ',
       'Тип действующего/ снятого ПУ', '№ установленного ПУ',
       'Марка установленного ПУ', 'Номер пломбы',
       'Показание действующего / снятого ПУ', 'Показание установленного ПУ',
       'Ссылка на контент', 'Координаты', '№ SIM карты',
       'Номер выданного пульта', 'Номер установленного ТТ',
       'Номер установленного ТН', 'Техническое решение по ФЗ 522', 'Опуск',
       'Номинал выключателя', 'Вид связи'],
      dtype='object')

In [9]:
df.rename(columns={
    '№ п/п': 'number', 'Филиал': 'filial', 'РЭС': 'res', 'Дата выполнения задания': 'date_done', 'Дата подтверждения задания Подрядчиком': 'date_proof_sek', 'Дата подтверждения задания РМР': 'date_proof_rmr',
    'Дата отклонения задания РМР': 'date_not_proof_rmr', 'Пользователь в "Цифровой контролёр"': 'user', 'Категория потребителей': 'category_partners', 'Вид работ:': 'vid_rabot', 'Объект работ': 'object', 'Адрес потребителя': 'address',
    'Сетевой адрес': 'net_address', 'ФИО': 'fio', 'Номер заявки SAP': 'sap_number', 'Дата заяки SAP': 'date_sap', 'Дата поступления заявки в МЛ': 'date_ml', 'Текущий статус\n': 'status',
    'Причина отклонения': 'reason_not_proof', '№ действующего/ снятого ПУ': 'serial_number_old', 'Тип действующего/ снятого ПУ': 'type_pu_old', '№ установленного ПУ': 'serial_new', 'Марка установленного ПУ': 'type_new', 'Номер пломбы': 'seal',
    'Показание действующего / снятого ПУ': 'pokaz_old', 'Показание установленного ПУ': 'pokaz_new', 'Ссылка на контент': 'ssylka', 'Координаты': 'koord', '№ SIM карты': 'sim_number', 'Номер выданного пульта': 'number_pult',
    'Номер установленного ТТ': 'tt_new', 'Номер установленного ТН': 'tn_new', 'Техническое решение по ФЗ 522': 'tech_resh_522', 'Опуск': 'opusk', 'Номинал выключателя': 'num_off', 'Вид связи': 'vid_svyaz'
}, inplace=True)

In [10]:
mass.rename(columns={'Номер заявки': 'sap_number'}, inplace=True)

In [11]:
mass['mass_zaves'] = 1

In [12]:
mass.head()

,sap_number,mass_zaves
0,И-22-00-599522/522/Ю8,1
1,И-22-00-599525/522/Ю8,1
2,И-22-00-599526/522/Ю8,1
3,И-22-00-599528/522/Ю8,1
4,И-22-00-599529/522/Ю8,1


In [13]:
df = df.merge(mass, on='sap_number', how='left')

In [14]:
df.shape

(94229, 37)

In [15]:
# Общее количество
df_all = df[df.user.str.contains('522')] \
    .groupby('filial', as_index=False) \
    .agg({'number': 'count', 'mass_zaves': 'count'}) \
    .rename(columns={'number': 'done', 'mass_zaves': 'mass_zaves_all'})
df_all['notmass_zaves_all'] = df_all['done'] - df_all['mass_zaves_all']
df_all

,filial,done,mass_zaves_all,notmass_zaves_all
0,ВЭС,15750,7678,8072
1,ЗЭС,19441,11627,7814
2,МВС,4,0,4
3,МКС,9333,75,9258
4,НМЭС,537,3,534
5,СЭС,19807,13028,6779
6,ЮЭС,27710,12721,14989


In [16]:
# Подтверждено РМР
df_rmr = df[(df.user.str.contains('522')) & (df.status == 'Подтверждено РМР')] \
    .groupby('filial', as_index=False) \
    .agg({'number': 'count', 'mass_zaves': 'count'}) \
    .rename(columns={'number': 'proof_rmr', 'mass_zaves': 'mass_zaves_rmr'})
df_rmr['notmass_zaves_rmr'] = df_rmr['proof_rmr'] - df_rmr['mass_zaves_rmr']
df_rmr

,filial,proof_rmr,mass_zaves_rmr,notmass_zaves_rmr
0,ВЭС,13513,6229,7284
1,ЗЭС,18223,11367,6856
2,МВС,4,0,4
3,МКС,8573,75,8498
4,НМЭС,506,3,503
5,СЭС,16988,10923,6065
6,ЮЭС,26671,12437,14234


In [17]:
# Подтверждено подрядчиком
df_rmr_0 = df[(df.user.str.contains('522')) & (df.status == 'Подтверждено подрядчиком')] \
    .groupby('filial', as_index=False) \
    .agg({'number': 'count',
          'mass_zaves': 'count'}) \
    .rename(columns={'number': 'wait_rmr', 'mass_zaves': 'mass_zaves_rmr_0'})
df_rmr_0['notmass_zaves_rmr_0'] = df_rmr_0['wait_rmr'] - df_rmr_0['mass_zaves_rmr_0']
df_rmr_0

,filial,wait_rmr,mass_zaves_rmr_0,notmass_zaves_rmr_0
0,ВЭС,55,36,19
1,ЗЭС,109,12,97
2,МКС,52,0,52
3,НМЭС,1,0,1
4,СЭС,280,214,66
5,ЮЭС,328,19,309


In [18]:
# На подтверждении у СЭК
df_sek = df[(df.user.str.contains('522')) & (df.status == 'На подтверждении')] \
    .groupby('filial', as_index=False) \
    .agg({'number': 'count',
          'mass_zaves': 'count'}) \
    .rename(columns={'number': 'wait_sek', 'mass_zaves': 'mass_zaves_sek'})
df_sek['notmass_zaves_sek'] = df_sek['wait_sek'] - df_sek['mass_zaves_sek']
df_sek

,filial,wait_sek,mass_zaves_sek,notmass_zaves_sek
0,ВЭС,1928,1224,704
1,ЗЭС,841,142,699
2,МКС,214,0,214
3,СЭС,1720,1329,391
4,ЮЭС,280,77,203


In [19]:
# Отклонено РМР
df_rmr_not_proof = df[(df.user.str.contains('522')) & (df.status == 'Отклонено РМР')] \
    .groupby('filial', as_index=False) \
    .agg({'number': 'count',
          'mass_zaves': 'count'}) \
    .rename(columns={'number': 'not_proof_rmr', 'mass_zaves': 'not_proof_rmr_mass_zaves'})
df_rmr_not_proof['not_proof_rmr_not_mass'] = df_rmr_not_proof['not_proof_rmr'] - df_rmr_not_proof['not_proof_rmr_mass_zaves']
df_rmr_not_proof

,filial,not_proof_rmr,not_proof_rmr_mass_zaves,not_proof_rmr_not_mass
0,ВЭС,55,49,6
1,ЗЭС,105,18,87
2,МКС,204,0,204
3,СЭС,184,131,53
4,ЮЭС,139,52,87


In [20]:
# Забраковано
df_delete = df[(df.user.str.contains('522')) & (df.status.str.contains('Забраковано'))] \
    .groupby('filial', as_index=False) \
    .agg({'number': 'count',
          'mass_zaves': 'count'}) \
    .rename(columns={'number': 'delete', 'mass_zaves': 'mass_zaves_delete'})
df_delete['notmass_zaves_delete'] = df_delete['delete'] - df_delete['mass_zaves_delete']
df_delete

,filial,delete,mass_zaves_delete,notmass_zaves_delete
0,ВЭС,199,140,59
1,ЗЭС,163,88,75
2,МКС,290,0,290
3,НМЭС,30,0,30
4,СЭС,635,431,204
5,ЮЭС,292,136,156


In [21]:
lst = [df_rmr, df_rmr_0, df_sek, df_rmr_not_proof, df_delete]

In [22]:
s = df_all
for i in lst:
    s = s.merge(i, on='filial', how='left')

In [23]:
s = s.fillna(0)

In [24]:
for i in s.columns:
    if i != 'filial':
        s[i] = s[i].astype(int)

In [25]:
s

,filial,done,mass_zaves_all,notmass_zaves_all,proof_rmr,mass_zaves_rmr,notmass_zaves_rmr,wait_rmr,mass_zaves_rmr_0,notmass_zaves_rmr_0,wait_sek,mass_zaves_sek,notmass_zaves_sek,not_proof_rmr,not_proof_rmr_mass_zaves,not_proof_rmr_not_mass,delete,mass_zaves_delete,notmass_zaves_delete
0,ВЭС,15750,7678,8072,13513,6229,7284,55,36,19,1928,1224,704,55,49,6,199,140,59
1,ЗЭС,19441,11627,7814,18223,11367,6856,109,12,97,841,142,699,105,18,87,163,88,75
2,МВС,4,0,4,4,0,4,0,0,0,0,0,0,0,0,0,0,0,0
3,МКС,9333,75,9258,8573,75,8498,52,0,52,214,0,214,204,0,204,290,0,290
4,НМЭС,537,3,534,506,3,503,1,0,1,0,0,0,0,0,0,30,0,30
5,СЭС,19807,13028,6779,16988,10923,6065,280,214,66,1720,1329,391,184,131,53,635,431,204
6,ЮЭС,27710,12721,14989,26671,12437,14234,328,19,309,280,77,203,139,52,87,292,136,156


In [26]:
df2 = df[(df.user.str.contains('522')) & (df.status == 'На подтверждении')]

In [27]:
df2.shape

(4983, 37)

In [28]:
df2['month_year'] = df2['date_done'].dt.to_period('M')

C:\Users\alaev\AppData\Local\Temp\ipykernel_5064\3759956365.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['month_year'] = df2['date_done'].dt.to_period('M')


In [29]:
z = df2.groupby(['filial', 'month_year'], as_index=False) \
    .agg({'number': 'count'}) \
    .rename(columns={'number': 'count'}) \
    .sort_values('month_year') \
    .pivot(index='filial', columns='month_year', values='count').fillna('')

In [30]:
z

month_year,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01
filial,,,,,,,,,,,
ВЭС,1.0,5.0,26.0,3.0,20.0,46.0,83.0,180.0,541.0,486.0,537.0
ЗЭС,1.0,,,,,2.0,,41.0,3.0,7.0,787.0
МКС,,2.0,1.0,,2.0,5.0,4.0,5.0,18.0,20.0,157.0
СЭС,1.0,2.0,3.0,10.0,30.0,24.0,66.0,57.0,276.0,763.0,488.0
ЮЭС,,9.0,,,,,8.0,10.0,27.0,99.0,127.0


In [31]:
# Запись двух DataFrame в один файл Excel с разными вкладками
with pd.ExcelWriter('output.xlsx') as writer:
    s.to_excel(writer, sheet_name='Sheet1', index=False)
    z.to_excel(writer, sheet_name='Sheet2', index=False)


In [32]:
token = '6861664837:AAGu1xA6-jbZ6j8EOVSNZw1MrtCmUmiQzyg'
chat_id = 1849097867

filepath = 'C:\\Users\\alaev\\report_sek\\output.xlsx'

params = {'chat_id': chat_id}
base_url = f'https://api.telegram.org/bot{token}/'
url = base_url + 'sendDocument?' + urlencode(params)

files = {'document': open(filepath, 'rb')}


resp = requests.get(url, files=files)